In [1]:
media_log = "eth0_audience_video4.csv"
rexfer_log = "eth0_video4_arq_res.csv"
request_log = "eth0_video_rex_req_v3.csv"

In [2]:
import pandas as pd

media_df = pd.read_csv(media_log, index_col=False)
rexfer_df = pd.read_csv(rexfer_log, index_col=False)
request_df = pd.read_csv(request_log, index_col=False)

In [3]:
media_df.describe()

,No.,Time,frame,subseq,seq,ts,pkt_cnt,stream_type,frame_type
count,17393.000000,17393.000000,17393.000000,17393.000000,17393.000000,17393.000000,17393.000000,17393.0,17393.000000
mean,65002.306560,100.123133,2499.902087,2.569195,8696.000000,32150.003507,6.138389,0.0,0.979589
std,40232.889549,58.708924,889.368469,2.255487,5021.070952,18853.393983,2.468033,0.0,0.141404
min,87.000000,1.058526,1001.000000,0.000000,0.000000,3.000000,3.000000,0.0,0.000000
25%,29841.000000,49.099490,1727.000000,1.000000,4348.000000,15442.000000,5.000000,0.0,1.000000
50%,64798.000000,100.140137,2500.000000,2.000000,8696.000000,31539.000000,6.000000,0.0,1.000000
75%,99651.000000,150.889315,3269.000000,4.000000,13044.000000,48513.000000,6.000000,0.0,1.000000
max,135221.000000,201.845779,4041.000000,31.000000,17392.000000,65532.000000,32.000000,0.0,1.000000


In [4]:
media_df[media_df[["seq"]].duplicated()].count()

No.            0
Time           0
Source         0
Destination    0
Protocol       0
frame          0
subseq         0
seq            0
ts             0
pkt_cnt        0
stream_type    0
frame_type     0
dtype: int64

In [5]:
media_df[["frame_type"]].value_counts()

frame_type
1             17038
0               355
dtype: int64

video4汇总信息：17393 条目，时间范围 (1.058, 201.845)秒，包seq没有出现wrap around（因此seq可以作为每个包的标识），每一帧最小分包数是3，最大分包数是32，发送一直为大流，其中2.04%个包是I帧，97.96%个分包是Delta帧。

In [6]:
request_df.describe()

,No.,Time,count,req_ms,losts_extend
count,6899.000000,6899.000000,6899.000000,6.899000e+03,0.0
mean,75643.494709,100.007275,5.749819,5.769940e+06,NaN
std,42579.645455,55.100490,3.917017,5.510067e+04,NaN
min,1560.000000,4.569871,1.000000,5.674503e+06,NaN
25%,38848.000000,52.332370,3.000000,5.722266e+06,NaN
50%,75499.000000,99.825424,5.000000,5.769759e+06,NaN
75%,111948.000000,147.190216,8.000000,5.817124e+06,NaN
max,149256.000000,195.741727,31.000000,5.865671e+06,NaN


In [7]:
rexfer_df.describe()

,No.,Time,frame,subseq,seq,req_ms,ts,pkt_cnt,stream_type,frame_type,losts_extend
count,39657.000000,39657.000000,39657.000000,39657.000000,39657.000000,3.965700e+04,39657.000000,39657.000000,39657.0,39657.000000,0.0
mean,76198.163603,100.719478,1589.544923,2.540560,9228.954434,5.770618e+06,32042.134352,6.094309,0.0,0.999017,NaN
std,42363.002094,54.819490,828.678573,1.923196,4805.738834,5.481943e+04,18771.723257,1.428330,0.0,0.031345,NaN
min,1575.000000,4.603560,142.000000,0.000000,822.000000,5.674503e+06,2.000000,2.000000,0.0,0.000000,NaN
25%,39807.000000,53.414800,873.000000,1.000000,5075.000000,5.723318e+06,15894.000000,5.000000,0.0,1.000000,NaN
50%,76378.000000,100.965437,1594.000000,2.000000,9247.000000,5.770867e+06,31395.000000,6.000000,0.0,1.000000,NaN
75%,113162.000000,148.733188,2314.000000,4.000000,13438.000000,5.818633e+06,47648.000000,7.000000,0.0,1.000000,NaN
max,149284.000000,195.771448,3029.000000,14.000000,17583.000000,5.865671e+06,65509.000000,15.000000,0.0,1.000000,NaN


In [8]:
total_request_times = request_df["count"].sum()
total_rexfer_times = rexfer_df.shape[0]
print(f"总请求次数: {total_request_times}")
print(f"总重传次数: {total_rexfer_times}")
print(f"重传率: {total_rexfer_times/total_request_times:.3%}")

总请求次数: 39668
总重传次数: 39657
重传率: 99.972%


In [9]:
import re


# This function aims to parse "losts" string, and return a list of seqs.
def parseLosts(this_str, size):
    results = []
    pattern = re.compile(r"\d+")
    matched = pattern.findall(this_str)
    #assert (len(matched), size)
    base_seq = 0
    for i, m in enumerate(matched):
        if i == 0:
            base_seq = int(m)
        else:
            results.append(int(m) + base_seq)
    return results


# This function aims to parse "losts" string, separate a single entry to multiple entrys, and get a new dataframe.
def transform_request_df(this_df):
    new_dicts = []
    for request_msg in this_df.itertuples(name="request_msg"):
        #assert (request_msg.Protocol, "UDP1.VIDEO_REX_REQ_V3")

        truncated = 60
        results = []
        if request_msg.count > truncated:
            results += parseLosts(request_msg.losts, truncated)
            results += parseLosts(
                request_msg.losts_extend, request_msg.count - truncated
            )
        else:
            results += parseLosts(request_msg.losts, request_msg.count)

        for i in range(int(request_msg.count)):
            this_dict = {}
            this_dict["Time"] = request_msg.Time
            this_dict["Protocol"] = request_msg.Protocol
            this_dict["seq"] = results[i]
            new_dicts.append(this_dict)

    new_df = pd.DataFrame(new_dicts)
    return new_df


def transform_rexfer_df(this_df):
    new_df = this_df[["Time", "Protocol", "seq"]]
    return new_df

<>:9: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:23: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:9: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:23: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-9-490f0f81a80d>:9: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (len(matched), size)
<ipython-input-9-490f0f81a80d>:23: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (request_msg.Protocol, "UDP1.VIDEO_REX_REQ_V3")


In [10]:
df_rex = transform_rexfer_df(rexfer_df)
df_req = transform_request_df(request_df)

In [11]:
df_rex.describe()

,Time,seq
count,39657.000000,39657.000000
mean,100.719478,9228.954434
std,54.819490,4805.738834
min,4.603560,822.000000
25%,53.414800,5075.000000
50%,100.965437,9247.000000
75%,148.733188,13438.000000
max,195.771448,17583.000000


In [12]:
df_req.describe()

,Time,seq
count,39668.000000,39668.000000
mean,100.684601,9228.900373
std,54.821292,4805.915486
min,4.569871,822.000000
25%,53.307666,5071.000000
50%,100.934125,9247.000000
75%,148.699497,13438.000000
max,195.741727,17583.000000


In [13]:
df_combined = pd.concat([df_req, df_rex], ignore_index=True)
df_combined.sort_values(
    by=["Time", "seq"], axis=0, ascending=[True, True], inplace=True
)
df_combined.reset_index(drop=True, inplace=True)
df_combined.to_csv("req_and_rex.csv")

In [14]:
import simpy


class PacerQueue:
    def __init__(self, this_df):
        self.env = simpy.Environment()
        self.length = 0
        self.df = this_df
        self.trace = []

    def change_queue(self, timestamp, protocol):
        yield self.env.timeout(timestamp)
        if protocol == "UDP1.VIDEO_REX_REQ_V3":
            self.length += 1
            self.trace.append({'Time':timestamp,'Length':self.length})
        elif protocol == "UDP1.VIDEO4_ARQ_RES":
            self.length -= 1
            self.trace.append({'Time':timestamp,'Length':self.length})
        
    def run(self):
        for log in self.df.itertuples(name="log"):
            self.env.process(self.change_queue(log.Time,log.Protocol))
        self.env.run()
        new_trace = pd.DataFrame(self.trace)
        return new_trace
    
simQueue = PacerQueue(df_combined)
df_trace = simQueue.run()
df_trace.to_csv("queue_length.csv")

In [15]:
df_trace.describe()

,Time,Length
count,79325.000000,79325.000000
mean,100.702037,14.318197
std,54.820048,5.846855
min,4.569871,0.000000
25%,53.343382,10.000000
50%,100.945592,14.000000
75%,148.699497,18.000000
max,195.771448,47.000000
